# Process results

This notebook analyzes the matching decisions by the simulation.py and summarizes a few metrics

In [5]:
import pandas as pd
import numpy as np

In [6]:
recipients = pd.read_csv('data/anon_recipients.csv',index_col=0,parse_dates=['date'])
recipients = recipients.set_index(['rec_id','date']).stack().to_frame().reset_index()
recipients.columns=['rec_id','date','ppe','qty']
recipients = recipients[recipients['qty']>0]

In [7]:
recipients

,rec_id,date,ppe,qty
3,rec0,2020-04-02 16:27:00+00:00,nitrileGloves,10000.0
9,rec0,2020-04-02 16:27:00+00:00,respirators,10000.0
10,rec0,2020-04-02 16:27:00+00:00,gowns,1000.0
12,rec0,2020-04-02 16:27:00+00:00,faceShields,5000.0
19,rec1,2020-04-02 16:35:00+00:00,disinfectingWipes,4.0
...,...,...,...,...
114381,rec6354,2020-07-17 08:14:58.249000+00:00,respirators,50.0
114389,rec6354,2020-07-17 08:14:58.249000+00:00,rec_req_id,6354.0
114395,rec6355,2020-07-17 14:25:54.605000+00:00,disposableBooties,1.0
114399,rec6355,2020-07-17 14:25:54.605000+00:00,respirators,1.0


In [12]:
donors = pd.read_csv('data/anon_donors.csv',index_col=0,parse_dates=['date'])
donors

,don_id,date,ppe,qty,don_req_id
469,don0,2020-04-09 13:08:00+00:00,faceShields,10.0,0
470,don1,2020-04-09 13:36:00+00:00,faceShields,1.0,1
327,don2,2020-04-09 13:53:00+00:00,faceShields,3000.0,2
328,don3,2020-04-09 14:24:00+00:00,faceShields,5.0,3
329,don71,2020-04-09 17:52:00+00:00,faceShields,10.0,4
...,...,...,...,...,...
20,don994,2020-07-17 21:52:59.717000+00:00,respirators,500.0,992
1658,don994,2020-07-17 21:52:59.717000+00:00,faceShields,500.0,993
13,don994,2020-07-17 21:52:59.717000+00:00,coveralls,1000.0,994
1708,don995,2020-07-18 12:11:44.881000+00:00,handmadeMasks,50.0,995


In [13]:
distance = pd.read_csv('data/anon_distance_matrix.csv',index_col=0)
distance

C:\Users\msamorani\AppData\Roaming\Python\Python36\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,don_id,rec_id,distance
0,don585,rec4650,540.263969
1,don749,rec5876,770.589552
2,don749,rec6288,1017.043444
3,don749,rec1454,612.656722
4,don749,rec472,1737.009898
...,...,...,...
2782715,don569,rec4976,883.147945
2782716,don984,rec4649,2452.256239
2782717,don984,rec5801,873.115576
2782718,don569,rec4649,1714.408610


In [7]:
decisions = pd.read_csv('decisions.csv',index_col=0,parse_dates=['date'])
decisions

,date,don_id,rec_id,ppe,qty,distance
0,2020-04-09 16:26:00+00:00,don0,rec0,faceShields,10.0,2548.016134
1,2020-04-09 16:26:00+00:00,don1,rec1,faceShields,1.0,2527.163615
2,2020-04-09 16:26:00+00:00,don3,rec10,faceShields,5.0,2067.043889
3,2020-04-16 16:26:00+00:00,don10,rec0,faceShields,20.0,2403.930303
4,2020-04-16 16:26:00+00:00,don11,rec1,faceShields,8.0,1865.637084
...,...,...,...,...,...,...
802,2020-07-16 16:26:00+00:00,don967,rec1107,handmadeMasks,100.0,992.490911
803,2020-07-16 16:26:00+00:00,don957,rec3493,coveralls,5.0,360.151787
804,2020-07-16 16:26:00+00:00,don962,rec1000,handSanitizer,3.0,2546.530683
805,2020-07-16 16:26:00+00:00,don955,rec0,respirators,5.0,37.010205


In [8]:
decisions.groupby('don_id').size().sort_values()

don_id
don0      1
don637    1
don638    1
don639    1
don64     1
         ..
don609    3
don625    3
don494    4
don824    4
don835    5
Length: 719, dtype: int64

In [10]:
ppes_to_consider = set(donors.ppe.unique())
ppes_to_consider = ppes_to_consider.intersection(set(recipients.ppe.unique()))

# For each ppe, compute stats

Fill rate

In [11]:
merg = recipients.merge(decisions,on=['rec_id','ppe'],how='left',suffixes=('_rec','_decision'))
merg

,rec_id,date_rec,ppe,qty_rec,date_decision,don_id,qty_decision,distance
0,rec0,2020-04-02 16:27:00+00:00,nitrileGloves,10000.0,2020-04-23 16:26:00+00:00,don100,100.0,2439.442188
1,rec0,2020-04-02 16:27:00+00:00,nitrileGloves,10000.0,2020-04-30 16:26:00+00:00,don372,100.0,1155.276088
2,rec0,2020-04-02 16:27:00+00:00,nitrileGloves,10000.0,2020-05-07 16:26:00+00:00,don544,100.0,1582.607709
3,rec0,2020-04-02 16:27:00+00:00,nitrileGloves,10000.0,2020-05-14 16:26:00+00:00,don697,4.0,2406.387041
4,rec0,2020-04-02 16:27:00+00:00,nitrileGloves,10000.0,2020-05-21 16:26:00+00:00,don774,100.0,72.754936
...,...,...,...,...,...,...,...,...
37792,rec6354,2020-07-17 08:14:58.249000+00:00,respirators,50.0,NaT,NaN,NaN,NaN
37793,rec6354,2020-07-17 08:14:58.249000+00:00,rec_req_id,6354.0,NaT,NaN,NaN,NaN
37794,rec6355,2020-07-17 14:25:54.605000+00:00,disposableBooties,1.0,NaT,NaN,NaN,NaN
37795,rec6355,2020-07-17 14:25:54.605000+00:00,respirators,1.0,NaT,NaN,NaN,NaN


In [12]:
merg['delay'] = (merg['date_decision'] - merg['date_rec']).dt.days
#merg = merg[merg['delay'] < threshold_days]

In [13]:
merg

,rec_id,date_rec,ppe,qty_rec,date_decision,don_id,qty_decision,distance,delay
0,rec0,2020-04-02 16:27:00+00:00,nitrileGloves,10000.0,2020-04-23 16:26:00+00:00,don100,100.0,2439.442188,20.0
1,rec0,2020-04-02 16:27:00+00:00,nitrileGloves,10000.0,2020-04-30 16:26:00+00:00,don372,100.0,1155.276088,27.0
2,rec0,2020-04-02 16:27:00+00:00,nitrileGloves,10000.0,2020-05-07 16:26:00+00:00,don544,100.0,1582.607709,34.0
3,rec0,2020-04-02 16:27:00+00:00,nitrileGloves,10000.0,2020-05-14 16:26:00+00:00,don697,4.0,2406.387041,41.0
4,rec0,2020-04-02 16:27:00+00:00,nitrileGloves,10000.0,2020-05-21 16:26:00+00:00,don774,100.0,72.754936,48.0
...,...,...,...,...,...,...,...,...,...
37792,rec6354,2020-07-17 08:14:58.249000+00:00,respirators,50.0,NaT,NaN,NaN,NaN,NaN
37793,rec6354,2020-07-17 08:14:58.249000+00:00,rec_req_id,6354.0,NaT,NaN,NaN,NaN,NaN
37794,rec6355,2020-07-17 14:25:54.605000+00:00,disposableBooties,1.0,NaT,NaN,NaN,NaN,NaN
37795,rec6355,2020-07-17 14:25:54.605000+00:00,respirators,1.0,NaT,NaN,NaN,NaN,NaN


In [14]:
res = merg.groupby(['rec_id','ppe']).agg({'qty_rec':min,'qty_decision':sum}).reset_index()
res['total_fill_rate'] = res.qty_decision / res.qty_rec
res

,rec_id,ppe,qty_rec,qty_decision,total_fill_rate
0,rec0,faceShields,5000.0,3557.0,0.7114
1,rec0,gowns,1000.0,501.0,0.5010
2,rec0,nitrileGloves,10000.0,414.0,0.0414
3,rec0,respirators,10000.0,3908.0,0.3908
4,rec1,disinfectingWipes,4.0,4.0,1.0000
...,...,...,...,...,...
37299,rec998,respirators,50.0,0.0,0.0000
37300,rec998,thermometers,2.0,0.0,0.0000
37301,rec999,faceShields,20.0,0.0,0.0000
37302,rec999,rec_req_id,999.0,0.0,0.0000


In [15]:
res

,rec_id,ppe,qty_rec,qty_decision,total_fill_rate
0,rec0,faceShields,5000.0,3557.0,0.7114
1,rec0,gowns,1000.0,501.0,0.5010
2,rec0,nitrileGloves,10000.0,414.0,0.0414
3,rec0,respirators,10000.0,3908.0,0.3908
4,rec1,disinfectingWipes,4.0,4.0,1.0000
...,...,...,...,...,...
37299,rec998,respirators,50.0,0.0,0.0000
37300,rec998,thermometers,2.0,0.0,0.0000
37301,rec999,faceShields,20.0,0.0,0.0000
37302,rec999,rec_req_id,999.0,0.0,0.0000


In [16]:
res.groupby('ppe')['total_fill_rate'].mean()

ppe
babyMonitors         0.005947
bodyBags             0.010870
coveralls            0.019680
disinfectingWipes    0.001500
disposableBooties    0.003790
faceShields          0.023524
gowns                0.000647
handSanitizer        0.002087
handmadeMasks        0.031359
nitrileGloves        0.004065
paprShield           0.000000
rec_req_id           0.000000
respirators          0.005956
safetyGlasses        0.009957
safetyGoggles        0.010815
surgicalCaps         0.001439
surgicalMasks        0.010740
thermometers         0.000019
Name: total_fill_rate, dtype: float64